In [ ]:
import asyncio
from datetime import datetime
import os

from astropy.time import Time
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Legend
from bokeh.plotting import figure
import numpy as np
import pandas as pd
import tables as tb

from lsst_efd_client import EfdClient

output_notebook()

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200128")
date_str = "20200129_010709"

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")

# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]

In [ ]:
datetimes = []
for key in keys:
    datetimes.append(datetime.strptime(key.replace('DT_', ''), '%Y%m%d_%H%M%S'))
datetimes = datetimes[2:-3]

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time(datetimes[0])
end = Time(datetimes[-1])
df = await client.select_time_series('lsst.sal.DSM.domeSeeing', '*', start.tai, end.tai, 1)
df_w = await client.select_time_series('lsst.sal.Environment.windSpeed', '*', start.tai, end.tai, 1)

In [ ]:
source = ColumnDataSource(df)
source_w = ColumnDataSource(df_w)
dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
tick_rotation = 75. * np.pi / 180.

s1 = figure(title = "Centroids")
cx = s1.line(x='index', y='centroidX', source=source, line_color='blue', line_width=2)
cy = s1.line(x='index', y='centroidY', source=source, line_color='black', line_width=2)
s1.xaxis.formatter = DatetimeTickFormatter(**formats)
s1.xaxis.major_label_orientation = tick_rotation
legend1 = Legend(items=[
    ("X" , [cx]),
    ("Y" , [cy]),
], location="center")
s1.add_layout(legend1, 'right')
s1.legend.margin = 0
s1.legend.padding = 0
s2 = figure(title='Seeing')
sx = s2.line(x='index', y='rmsX', source=source, line_color='blue', line_width=2)
sy = s2.line(x='index', y='rmsY', source=source, line_color='black', line_width=2)
s2.xaxis.formatter = DatetimeTickFormatter(**formats)
s2.xaxis.major_label_orientation = tick_rotation
s2.yaxis.axis_label = 'scaled RMS (arcseconds)'
legend2 = Legend(items=[
    ("X" , [sx]),
    ("Y" , [sy]),
], location="center")
s2.add_layout(legend2, 'right')
s2.legend.margin = 0
s2.legend.padding = 0
s3 = figure(title='Wind')
s3.line(x='index', y='avg2M', source=df_w, line_width=2)
s3.xaxis.formatter = DatetimeTickFormatter(**formats)
s3.xaxis.major_label_orientation = tick_rotation
s3.yaxis.axis_label = 'speed (m/s)'
s4 = figure(title='Flux')
s4.line(x='index', y='flux', source=df, line_width=2)
s4.xaxis.formatter = DatetimeTickFormatter(**formats)
s4.xaxis.major_label_orientation = tick_rotation
s5 = figure(title='Max ADC')
s5.line(x='index', y='maxADC', source=df, line_width=2)
s5.xaxis.formatter = DatetimeTickFormatter(**formats)
s5.xaxis.major_label_orientation = tick_rotation
s6 = figure(title='FWHM')
s6.line(x='index', y='fwhm', source=df, line_width=2)
s6.xaxis.formatter = DatetimeTickFormatter(**formats)
s6.xaxis.major_label_orientation = tick_rotation
s6.yaxis.axis_label = '(pixels)'
grid = gridplot([[s1, s2, s3], [s4, s5, s6]], plot_width=300, plot_height=300)
show(grid)